In [1]:
import csv
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import defaultdict
import os
from dotenv import load_dotenv

In [2]:
# Fetch environment variables
load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

# Set up Spotify API credentials
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
mil_tracks = pd.read_csv('spotify_data.csv')
known_tracks = pd.read_csv('intermediate.csv')
id = "1lzr43nnXAijIGYnCT8M8H"
found1 = mil_tracks[mil_tracks['track_id'] == id]
id = "spotify:track:" + id
found2 = known_tracks.loc[known_tracks['track_id'] == id]
if not found1.empty:
    print(found1)
if not found2.empty:
    print(found2)

In [6]:
import time
# ---------- YOUR FILEPATHS HERE ----------
FILENAME = "mpd.slice.8000-8999.json"
FOLDER = r"spotify_million_playlist_dataset/data"
PATH = FOLDER + "\\" + FILENAME
# ---------- YOUR FILEPATHS HERE ----------

# fields = ['','artist_name','track_name','track_id','popularity','year','genre','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
# no popularity,year,genre fields, not queryable in spotipy api
features = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
track_list = []
# request_ids = []
with open(PATH, "r") as playlist_file:
    batch = json.load(playlist_file)
    i = 0
    for playlist in batch["playlists"]:
        for track in playlist["tracks"]:
            id = track["track_uri"][14:]
            # Initialize track dict with data from the playlist json
            track_info = {'':i,'artist_name':track['artist_name'],'track_name':track['track_name'],'track_id':id}
            # Check if we already have information
            f = known_tracks[known_tracks['track_id'] == track['track_uri']]
            if not f.empty:
                for feature in features:
                    track_info[feature] = f[feature]
                i += 1
                continue

            f = mil_tracks[mil_tracks['track_id'] == id]
            if not f.empty:
                for feature in features:
                    track_info[feature] = f[feature]
                i += 1
                continue

            # if len(request_ids) < 100:
            #     request_ids.append(id)
            #     continue
            # Fetch more relevant track data using Spotipy
            while True:
                try:
                    audio_features = sp.audio_features(id)[0]
                    if audio_features:  # Check if audio_features is not None
                        for feature in features:
                            track_info[feature] = audio_features[feature]
                        track_list.append(track_info)
                        i += 1
                    time.sleep(2)  # Pause to respect rate limits
                    break  # Exit the retry loop on success
                except spotipy.exceptions.SpotifyException as e:
                    if e.http_status == 429:
                        retry_after = int(e.headers.get('Retry-After', 1))
                        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                        time.sleep(retry_after+1)
                    else:
                        print(f"Spotify API error: {e}")
                        break  # Exit the retry loop on non-rate limit errors
                except Exception as e:
                    print(f"An error occurred: {e}")
                    break  # Exit the retry loop on other errors

Max Retries reached


Rate limit exceeded. Retrying after 1 seconds...


KeyboardInterrupt: 

In [5]:
print(len(track_list))
print(track_list[0])

0


IndexError: list index out of range

In [15]:
# Write back data into CSV file, similar style as the million track dataset
fields = ['','artist_name','track_name','track_id']
fields = fields + features
OUTFILE = FILENAME[:-5] + ".csv" # csv version of input json file
with open(OUTFILE, "w", encoding="utf-8") as output:
    writer = csv.DictWriter(output, fieldnames=fields)
    writer.writeheader()
    writer.writerows(track_list)